In [1]:
from common import *
from matrix import *
from helper.weighted import *
%matplotlib inline
canvas(16, 12)
import pickle
import gc

/home/ec2-user/anaconda3/lib/python3.6/site-packages/pandas/core/common.py:701: RuntimeWarning: divide by zero encountered in log
  return func(obj, *args, **kwargs)


In [2]:
from models.thealg import TheAlg
from models.split import splitf

In [3]:
o = O()
def_W(o); W = o.Wa.astype(np.float32)
def_Y(o); Y = o.Ya.astype(np.float32)

#### x features {

In [4]:
temp = '5zombie'
#X.to_hdf('/big/data/ready/algrun' + temp, key='X')
X = pd.read_hdf(top_dir + 'data/ready/algrun' + temp, key='X')

In [5]:
mvalid = o.Y.count()

In [6]:
try:
    dict.clear(o)
    del o
except NameError:
    pass
gc.collect()
try:
    dict.clear(of)
    del of
except NameError:
    pass
gc.collect()

0

#### } x features end

In [7]:
oW, oX, oY, omvalid = W, X, Y, mvalid

##### data analysis

In [ ]:
# NEEDS thnigs below to be defined firsT!

In [ ]:
k, m = X.columns.get_level_values(0).nunique(), X.columns.get_level_values(1).nunique()
featnames = [str(X.columns[m * f][0]) for f in range(k)]
namesf = lambda k: tuple(featnames[j] for j in k)

In [ ]:
ii = 0
scores, fbnums = scores__[ii], fbnums__[ii]
sub = lambda scores, j: [(tc / scores[None][0], ho / scores[None][1], k)
        for k, (tc, ho) in scores.items() if k is not None if len(k) == j]
subf = lambda scores, fbnums, j: [(tc / scores[None][0], ho / scores[None][1],) + ((fbnums[k],) if fbnums else ()) +  (k,)
        for k, (tc, ho) in scores.items() if k is not None if len(k) == j]
#_ = [(print(line), print(namesf(line[-1])), print()) for line in sorted(sub)[:100]]

In [15]:
for group, (sfilt, alg, scores, fbnums, look) in enumerate(zip(sfilt__, alg__, scores__, fbnums__, look__)):
    a = [sub(scores, j) for j in range(0, 11)]
    jlen = [len(b) for b in a]
    tcavg = [round(sum(e[0] for e in b) / (len(b) + (len(b) == 0)) * 1e6) / 1e6 for b in a]
    hoavg = [round(sum(e[0] for e in b) / (len(b) + (len(b) == 0)) * 1e6) / 1e6 for b in a]
    print(jlen, ',')#, '\n', tcavg, '\n', hoavg)

[0, 14, 14, 14, 14, 14, 3, 0, 0, 0, 0] ,
[0, 14, 12, 12, 12, 10, 10, 5, 0, 0, 0] ,
[0, 17, 16, 15, 15, 15, 14, 13, 9, 3, 0] ,
[0, 17, 16, 16, 15, 9, 5, 3, 1, 1, 0] ,


##### search average algorithm

In [8]:
sub = lambda scores, j: [(tc / scores[None][0], ho / scores[None][1], k)
        for k, (tc, ho) in scores.items() if k is not None if len(k) == j]
subf = lambda scores, fbnums, j: [(tc / scores[None][0], ho / scores[None][1],) + ((fbnums[k],) if fbnums else ()) +  (k,)
        for k, (tc, ho) in scores.items() if k is not None if len(k) == j]

In [9]:
def _filter_search(scores):
    filted = {}
    bases = sorted(sub(scores, 1))
    for s1, _, k1 in bases:
        inclist = []
        s, k = s1, k1
        completed = False
        for j in range(2, 20):
            front = [tup for tup in sub(scores, j) if set(tup[-1]) > set(k)]
            front.sort()
            snew = front[0][0] / scores[None][0] if front else None
            if front and snew >= s:
                completed = True
                break
            elif not front:
                break
            inclist.append(k)
            k = front[0][-1]
            s = snew
        if completed:
            filted.update({k: scores[k] for k in inclist})
        else:
            break
    filted[None] = scores[None]
    return filted

In [10]:
temp = 'ghost'
savenames = ['[246&m3]{}', '[~246&m3]{}(0,2,4,8,16)(0,25,50,75,100)',
             '[249&~m3]{}', '[~249&~m3]{}(0,2,4,8,16)(0,25,50,75,100)']
featnames__, scores__, fbnums__ = [], [], []
for group, save in enumerate(savenames):
    with open(top_dir + 'data/progress/linsearch/{}/'.format(temp) + save.format(temp) + '.pkl', 'rb') as file:
        guys = pickle.load(file)
        featnames__.append(guys[0])
        scores__.append({(tuple(sorted(f)) if isinstance(f, tuple) else None): v for f, v in guys[1].items()})
        fbnums__.append({(tuple(sorted(f)) if isinstance(f, tuple) else None): v for f, v in guys[2].items()}
                        if len(guys) == 3 else None)
oscores__ = scores__
scores__ = [_filter_search(x) for x in oscores__]

In [11]:
lenfilter = lambda scores, k: sorted((tc, ho, f) for f, (tc, ho) in scores.items() if f and len(f) == k)
#lenfilterho = lambda scores, k: sorted((tc / 4 + ho, f) for f, (tc, ho) in scores.items() if f and len(f) == k)

In [12]:
def search_average(scores, look, use_ho=False):
    _lenfilter = lenfilter if not use_ho else lenfilterho
    from collections import Counter
    items = []
    if isinstance(look, list):
        look = {i: v for i, v in enumerate(look) if v != 0}
    for lenf, topn in look.items():
        items += _lenfilter(scores, lenf)[:topn]
    count = Counter()
    for things in items:
        f = things[-1]
        for j in f:
            count[j] += 1
    for things in items:
        f = things[-1]
        yield f, sum(1 / count[j] for j in f) / len(f)

In [13]:
sfilt__ = [
    stock_nvdf(dh, 246) & (stockmarket == 3),
    ~stock_nvdf(dh, 246) & (stockmarket == 3),
    stock_nvdf(dh, 249) & (stockmarket != 3),
    ~stock_nvdf(dh, 249) & (stockmarket != 3),
]
alg__ = ['canon', 'expfb', 'canon', 'expfb']
look__ = [
    [0, 14, 14, 14, 12, 0, 0, 0, 0, 0, 0],
    [0, 14, 13, 13, 12, 10, 5, 1, 0, 0, 0],
    [0, 16, 16, 15, 15, 15, 13, 4, 0, 0, 0],
    [0, 19, 18, 18, 13, 6, 0, 0, 0, 0, 0],
]
# ^ old subset data
look__ = [
    [0, 14, 14, 14, 14, 14,  3,  0, 0, 0, 0] ,
    [0, 14, 12, 12, 12, 10, 10,  5, 0, 0, 0] ,
    [0, 17, 16, 15, 15, 15, 14, 13, 9, 3, 0] ,
    [0, 17, 16, 16, 15,  9,  5,  3, 1, 1, 0] ,
]
look__ = [
    [0,  0,  0,  0,  0, 10,  0, 0, 0, 0, 0] ,
    [0,  0,  0,  0,  0,  0, 10, 0, 0, 0, 0] ,
    [0,  0,  0,  0,  0,  0,  0, 2, 9, 0, 0] ,
    [0,  0,  0,  0,  0,  9,  5, 0, 0, 0, 0] ,
]

In [14]:
DRY = False
SET = 'tc'

loop

In [15]:
yx__ = []
for group, (sfilt, alg, scores, fbnums, look) in enumerate(zip(sfilt__, alg__, scores__, fbnums__, look__)):
    print(group)
    W, X, Y, mvalid = oW, oX, oY, omvalid
    #
    W = W.T[sfilt].T
    Y = Y.T[sfilt].T
    X = X.wide_filter(sfilt).chain.fillna(0, inplace=True)
    mvalid = mvalid[sfilt]
    #
    s = splitf(X, 2, ho=False)
    ta = TheAlg(X, Y, W, s, mvalid, algorithm=alg, alltrain=True)
    #
    fw = list(search_average(scores, look, use_ho=False))
    feats_, args_, weight_ = [f for f, w in fw], [(fbnums[f] if fbnums else ()) for f, w in fw], [w for f, w in fw]
    yx = ta.final(feats_, args_, weight_, dry=DRY, average=True)
    yx__.append(yx + Y * 0)
Yx = pd.concat(yx__, axis=1).sort_index(axis=1)
W, X, Y, mvalid = oW, oX, oY, omvalid

0


/home/ec2-user/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3035: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


<10> 0.........
1


/home/ec2-user/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3035: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


<10> 0.........
2


/home/ec2-user/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3035: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


<11> 0.........10
3
<14> 0.........10...


fitting a nonnegative linear regression on the ys

In [ ]:
wideref = (dh.y.fillna(0) * 0 + 1 - 1).makesw().fillna(0)
def _make_matrix(y, sfilt):
    ret = wideref.copy()
    ret.loc[:, sfilt] = y
    return ret
    
yx__ = []
for group, (sfilt, alg, scores, fbnums, look) in enumerate(zip(sfilt__, alg__, scores__, fbnums__, look__)):
    print(group)
    W, X, Y, mvalid = oW, oX, oY, omvalid
    #
    W = W.T[sfilt].T
    Y = Y.T[sfilt].T
    X = X.wide_filter(sfilt).chain.fillna(0, inplace=True)
    mvalid = mvalid[sfilt]
    #
    s = split3(X, repeat=2, reverse=True)
    ta = TheAlg(X, Y, W, s, mvalid, algorithm=alg)
    #
    fw = list(search_average(scores, look, use_ho=False))
    feats_, args_, weight_ = [f for f, w in fw], [(fbnums[f][1] if fbnums else ()) for f, w in fw], [w for f, w in fw]
    yx = ta.final(feats_, args_, weight_, dry=DRY, average=False)
    #!#yx__.append(yx + Y * 0)
    yx__.append(pd.concat([_make_matrix(y).makeil_like(dh) for y in yx], axis=1))
#!#Yx = pd.concat(yx__, axis=1).sort_index(axis=1)
#!#W, X, Y, mvalid = oW, oX, oY, omvalid

In [155]:
yy__ = []
for i__, (yx, sfilt) in enumerate(zip(yx__, sfilt__)):
    print(i__)
    #for i in range(len(yx)):
    #    yx[i] = yx[i].makeil_like(dh)
    #yx__[i__] = pd.concat(yx, axis=1)
    ifilt = ifilt__[i__]
    yx = yx__[i__]
    s = split3(dh, repeat=2, reverse=True)
    from scipy.optimize import nnls
    beta, _ = nnls(yx[ifilt & s.tc].values * dh.w[ifilt & s.tc][:, np.newaxis] ** .5,
                   dh.y[ifilt & s.tc] * dh.w[ifilt & s.tc] ** .5)
    yy = yx @ beta
    yy[~ifilt] = 0
    yy__.append(yy)
iYy = sum(yy__)

0
1
2
3


In [156]:
iYy0, iY0, iW0 = (iYy[s[SET]], oY.makeil_like(dh)[s[SET]], oW.makeil_like(dh)[s[SET]])
((iYy0 - iY0) ** 2 * iW0).sum() / (iY0 ** 2 * iW0).sum()

0.63784580597848828

back to the classic averaging method

In [17]:
# conservatively deep 10
Yx0, Y0, W0 = Yx[s[SET]], Y[s[SET]], W[s[SET]]
((Yx0 - Y0) ** 2 * W0).sum().sum() / (Y0 ** 2 * W0).sum().sum()

0.56437814

In [19]:
iYx = Yx.makeil_like(dh)

(1264247,)

In [22]:
#np.save(top_dir + 'data/progress/linfinal/firstpick.npy', iYx)

final above, reverse split below

In [82]:
# deepest 20
Yx0, Y0, W0 = Yx[s[SET]], Y[s[SET]], W[s[SET]]
((Yx0 - Y0) ** 2 * W0).sum().sum() / (Y0 ** 2 * W0).sum().sum()

0.63487983

In [78]:
# deepest 10 non extremal (correct)
Yx0, Y0, W0 = Yx[s[SET]], Y[s[SET]], W[s[SET]]
((Yx0 - Y0) ** 2 * W0).sum().sum() / (Y0 ** 2 * W0).sum().sum()

0.63469827

In [75]:
# deepest 10 + 1 extremal guy on group 1  (MISTAKE on group 3!)
Yx0, Y0, W0 = Yx[s[SET]], Y[s[SET]], W[s[SET]]
((Yx0 - Y0) ** 2 * W0).sum().sum() / (Y0 ** 2 * W0).sum().sum()

0.63433874

In [71]:
# deepest 8 non extremal  (MISTAKE on group 3?)
Yx0, Y0, W0 = Yx[s[SET]], Y[s[SET]], W[s[SET]]
((Yx0 - Y0) ** 2 * W0).sum().sum() / (Y0 ** 2 * W0).sum().sum()

0.63485128

In [67]:
# deepest 10 non extremal (MISTAKE on group 3!)
Yx0, Y0, W0 = Yx[s[SET]], Y[s[SET]], W[s[SET]]
((Yx0 - Y0) ** 2 * W0).sum().sum() / (Y0 ** 2 * W0).sum().sum()

0.63429654

In [63]:
# deepest with at least 10 and 10 on the left beside it
Yx0, Y0, W0 = Yx[s[SET]], Y[s[SET]], W[s[SET]]
((Yx0 - Y0) ** 2 * W0).sum().sum() / (Y0 ** 2 * W0).sum().sum()

0.63526487

In [59]:
# deepest with at least 10
Yx0, Y0, W0 = Yx[s[SET]], Y[s[SET]], W[s[SET]]
((Yx0 - Y0) ** 2 * W0).sum().sum() / (Y0 ** 2 * W0).sum().sum()

0.63497686

In [55]:
# ~5 deepest
Yx0, Y0, W0 = Yx[s[SET]], Y[s[SET]], W[s[SET]]
((Yx0 - Y0) ** 2 * W0).sum().sum() / (Y0 ** 2 * W0).sum().sum()

0.63804036

In [52]:
# not more than 20 deepest
Yx0, Y0, W0 = Yx[s[SET]], Y[s[SET]], W[s[SET]]
((Yx0 - Y0) ** 2 * W0).sum().sum() / (Y0 ** 2 * W0).sum().sum()

0.63486272

In [49]:
# all (but no k=1,2,3)
Yx0, Y0, W0 = Yx[s[SET]], Y[s[SET]], W[s[SET]]
((Yx0 - Y0) ** 2 * W0).sum().sum() / (Y0 ** 2 * W0).sum().sum()

0.63538206

In [46]:
# all (but no k=1,2)
Yx0, Y0, W0 = Yx[s[SET]], Y[s[SET]], W[s[SET]]
((Yx0 - Y0) ** 2 * W0).sum().sum() / (Y0 ** 2 * W0).sum().sum()

0.63602185

In [43]:
# all (but no k=1)
Yx0, Y0, W0 = Yx[s[SET]], Y[s[SET]], W[s[SET]]
((Yx0 - Y0) ** 2 * W0).sum().sum() / (Y0 ** 2 * W0).sum().sum()

0.63711482

In [40]:
# all
Yx0, Y0, W0 = Yx[s[SET]], Y[s[SET]], W[s[SET]]
((Yx0 - Y0) ** 2 * W0).sum().sum() / (Y0 ** 2 * W0).sum().sum()

0.64153665

In [37]:
# one of k=(max)
Yx0, Y0, W0 = Yx[s[SET]], Y[s[SET]], W[s[SET]]
((Yx0 - Y0) ** 2 * W0).sum().sum() / (Y0 ** 2 * W0).sum().sum()

0.64866221

In [33]:
# one of k=4
Yx0, Y0, W0 = Yx[s[SET]], Y[s[SET]], W[s[SET]]
((Yx0 - Y0) ** 2 * W0).sum().sum() / (Y0 ** 2 * W0).sum().sum()

0.64920402

In [75]:
Y0, W0 = Y[s[SET]], W[s[SET]]
((Yx__combine - Y0) ** 2 * W0).sum().sum() / (Y0 ** 2 * W0).sum().sum()

0.67326117250855633

In [68]:
Y0, W0 = Y[s[SET]], W[s[SET]]
(((Yx__3of3 + Yx__bigtwo) / 2 - Y0) ** 2 * W0).sum().sum() / (Y0 ** 2 * W0).sum().sum()

0.67388656690142368

In [61]:
Y0, W0 = Y[s[SET]], W[s[SET]]
((Yx__bigtwo - Y0) ** 2 * W0).sum().sum() / (Y0 ** 2 * W0).sum().sum()

0.68432404158565274

In [67]:
Y0, W0 = Y[s[SET]], W[s[SET]]
((Yx__3of3 - Y0) ** 2 * W0).sum().sum() / (Y0 ** 2 * W0).sum().sum()

0.67563324448134887

In [43]:
Y0, W0 = Y[s[SET]], W[s[SET]]
((Yx__best - Y0) ** 2 * W0).sum().sum() / (Y0 ** 2 * W0).sum().sum()

0.68734226667352849

In [15]:
assert ta.columns.get_level_values(0).nunique() * ta.columns.get_level_values(1).nunique() == ta.columns.shape[0]
assert ta.k * ta.m == ta.columns.shape[0]

In [ ]:
submit(Yx, 'lineartest__combine')
old = pd.read_csv(top_dir + 'data/submit/model3.csv', index_col=0, dtype={'y': str})
old.y = (old.y.map(Decimal) * 10000).astype(float)
Old = old.makesw().fillna(0)
Old.columns = Old.columns.levels[1]
cut = lambda y_: y_[dg.y.makesw().columns]
submit(cut(Yx) * .5 + Old * .5, 'lineartest__combine_5050_model3___dec')

In [39]:
sfilt__ = [
    stock_nvdf(dh, 246) & (stockmarket == 3),
    ~stock_nvdf(dh, 246) & (stockmarket == 3),
    stock_nvdf(dh, 249) & (stockmarket != 3),
    ~stock_nvdf(dh, 249) & (stockmarket != 3),
]
ifilt__ = [
    index_nvdf(dh, 246) & (dh.Market == 3),
    ~index_nvdf(dh, 246) & (dh.Market == 3),
    index_nvdf(dh, 249) & (dh.Market != 3),
    ~index_nvdf(dh, 249) & (dh.Market != 3),
]
feats__ = [ #combine
    [(356, 54, 972, 185, 1), (340, 652, 119), (356, 54, 815)],
    [(652, 119, 980, 157), (652, 152, 356, 603), (656, 359, 152), (652, 152, 35), (656, 359, 49)],
    [(407, 1, 356, 291, 653, 651, 1006), (141, 356, 653, 651, 1), (407, 653, 192, 983, 1),
     (356, 653, 455), (35, 653, 196), (455, 653, 983)],
    [(407, 698, 196, 1, 327), (407, 698, 523, 977, 1), (455, 327, 1199), (407, 698, 1130), (1006, 698, 356)],
]
args__ = [ #combine
    [()] * len(feats__[0]),
    [(4, 75), (0, 100), (4, 75), (0, 100), (4, 75)],
    [()] * len(feats__[2]),
    [(0, 75), (0, 75), (0, 75), (0, 50), (0, 50)],
]
alg__ = ['canon', 'expfb', 'canon', 'expfb']
look__ = [
    {2:15, 3:30, 4:30, 5:15, 6:10, 7:5},
    {2:15, 3:30, 4:30, 5:15, 6:10, 7:5},
    {2:15, 3:30, 4:30, 5:15, 6:10, 7:5},
    {2:15, 3:30, 4:30, 5:15, 6:10, 7:5},
]

## diagnosis analysis

In [31]:
pname = '[~249&~m3]zombs(0,4,14,32)(25,50,75,100,125,150)'
featnames = [str(ta.columns[ta.m * f][0]) for f in range(ta.k)]

In [32]:
with open(top_dir + 'data/progress/linsearch/' + pname + '.pkl', 'wb') as file:
    pickle.dump((featnames, ta.scores, ta.fbnums), file, protocol=-1)

In [33]:
see = {tuple(featnames[j] for j in k): s for k, s in ta.scores.items() if k is not None}
tc0, ho0 = ta.scores[None]
sub = [(tc / tc0, ho / ho0, k) for k, (tc, ho) in see.items()
       if k is not None]
_ = [print(line) for line in sorted(sub)[:100]]

(0.652535685895395, 0.7004064900116118, ('xg.log.zc(-93).sgnint(03/44).wnrm', 'x3A.wnrm', 'x3E.log.z(1).window(<20>*!-).wnrm.pos', 'xg.log.zc(-93).sgnint(12/33).wnrm.neg'))
(0.6529830325397746, 0.6889513593543053, ('xg.log.zc(-93).sgnint(03/44).wnrm', 'x3A.wnrm', 'xg.log.zc(-93).sgnint(25/33).wnrm.neg', 'x3.log.zc(-93).sgnint(1/4).window(<20>*!-).wnrm.neg'))
(0.6530042363420493, 0.6926842767326568, ('xg.log.zc(-93).sgnint(03/44).wnrm', 'x3A.wnrm', 'x3.log.zc(-93).sgnint(0/4).window(<20>*!-).wnrm.neg', 'xg.log.zc(-93).sgnint(12/33).wnrm.neg'))
(0.6532938175117425, 0.6915756647607065, ('xg.log.zc(-93).sgnint(03/44).wnrm', 'x3A.wnrm', 'x3.log.zc(-93).sgnint(0/4).window(<20>*!-).wnrm.neg', 'xg.log.zc(-93).sgnint(03/12).wnrm.pos'))
(0.6534188969807563, 0.6888000588946109, ('xg.log.zc(-93).sgnint(03/44).wnrm', 'x3A.wnrm', 'x3.log.zc(-93).sgnint(0/4).window(<20>*!-).wnrm.neg', 'xg.log.zc(-93).sgnint(25/33).wnrm.neg'))
(0.6534188969807563, 0.6888000588946109, ('xg.log.zc(-93).sgnint(03/44).wnr